<a href="https://colab.research.google.com/github/josedavidcortes-com/finalComputerVision/blob/main/David_Cortes_Final_Exam_CNN_Computer_Vision_CSC752_UT1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
#lets get some libraries!
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from keras.models import Sequential
from keras.utils.np_utils import to_categorical

#CNN
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Dropout, Dense, Flatten, Input, Lambda

#tensorflow <3
import tensorflow as tf
from tensorflow import keras
import itertools

from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau , EarlyStopping

#those curly yellow underlines are a very known error, please check the next link for more information
#https://stackoverflow.com/questions/73358867/import-tensorflow-keras-optimizers-could-not-be-resolvedreportmissingimports
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers, losses


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

import numpy as np
import pandas as pd

#can I use my Christmas gift (NVIDIA GPU) to improve the time? Lets see!!
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as trans

#do we have a GPU? this is interesting, Im going to try this on local.
print('PyTorch Version: ', torch.__version__)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

PyTorch Version:  1.13.0+cu116


In [3]:
#lets load the dataset
#training folder path
train_dir = '/content/drive/MyDrive/shoe_dataset/train/'
#testing folder path
test_dir = '/content/drive/MyDrive/shoe_dataset/test/'

In [4]:
#Generate batches of tensor image data with real-time data augmentation. 
#Keras ImageDataGenerator is used for getting the input of the original data and further, it makes the transformation of 
#this data on a random basis and gives the output resultant containing only the data that is newly transformed. 
#It does not add the data. Keras image data generator class is also used to carry out data augmentation where we aim to gain 
#the overall increment in the generalization of the model. Operations such as rotations, translations, shearin, scale changes, 
# and horizontal flips are carried out randomly in data augmentation using an image data generator.
#for more information please check the next link:
#https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
trn_data = ImageDataGenerator( rescale = 1./ 255 , rotation_range = 35 , width_shift_range = 0.2, height_shift_range = 0.2 , shear_range = 0.2, zoom_range = 0.2 , horizontal_flip = True, fill_mode = 'nearest')

In [5]:
tst_data = ImageDataGenerator( rescale = 1./ 255 )

In [6]:

trn_dt = trn_data.flow_from_directory( directory = train_dir , batch_size = 32 , target_size = ( 240 , 240 ) , class_mode = "categorical" , shuffle = False)

tst_dt = tst_data.flow_from_directory( directory = test_dir  , batch_size = 32 , target_size = ( 240 , 240 ) , class_mode = "categorical")

Found 711 images belonging to 3 classes.
Found 114 images belonging to 3 classes.


In [7]:
model = tf.keras.models.Sequential([
    Conv2D( 16, (3,3) , activation = 'relu'  , input_shape = ( 240 , 240 , 3 ) ) , MaxPooling2D( 2 , 2 ),
    Conv2D( 32, (3,3) , activation = 'relu') , MaxPooling2D(2,2) ,
    Conv2D( 32, (3,3) , activation = 'relu') , MaxPooling2D(2,2) , Conv2D( 32, ( 3 , 3 ), activation = 'relu' ) , MaxPooling2D( 2 , 2 ) ,
    Flatten(),
    Dense( 512, activation = 'relu') , Dropout(0.2) , Dense( 3 , activation = 'softmax')
])

In [8]:
# Using the Adam Optimizer to set the learning rate of our final model
opt = optimizers.Adam(learning_rate=0.0001)

# Compiling and setting the parameters we want our model to use
model.compile( loss = "categorical_crossentropy" , optimizer = opt , metrics = ['accuracy'] )

In [ ]:
history = model.fit(trn_dt, epochs = 25 , steps_per_epoch = len(trn_dt) , validation_data = tst_dt , validation_steps = int ( 0.25 * len( tst_dt ) ) )

Epoch 1/25
23/23 [==============================] - 53s 2s/step - loss: 1.2483 - accuracy: 0.3558 - val_loss: 1.1539 - val_accuracy: 0.3438
Epoch 2/25
23/23 [==============================] - 48s 2s/step - loss: 1.1740 - accuracy: 0.3179 - val_loss: 1.1244 - val_accuracy: 0.2500
Epoch 3/25
23/23 [==============================] - 42s 2s/step - loss: 1.1129 - accuracy: 0.2925 - val_loss: 1.0962 - val_accuracy: 0.2812
Epoch 4/25
23/23 [==============================] - 42s 2s/step - loss: 1.1079 - accuracy: 0.3361 - val_loss: 1.0960 - val_accuracy: 0.3125
Epoch 5/25
23/23 [==============================] - 43s 2s/step - loss: 1.1025 - accuracy: 0.2911 - val_loss: 1.1050 - val_accuracy: 0.3438
Epoch 6/25
23/23 [==============================] - 44s 2s/step - loss: 1.0987 - accuracy: 0.3165 - val_loss: 1.1004 - val_accuracy: 0.3750
Epoch 7/25
23/23 [==============================] - 50s 2s/step - loss: 1.1049 - accuracy: 0.3263 - val_loss: 1.0925 - val_accuracy: 0.3438
Epoch 8/25
23/23 [==

In [ ]:
#Lets see if the layers of the model are correct
model.summary()

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
#setting the plot
plt.subplot( 2 , 1 , 2 )
plt.plot( loss , label = 'Training Loss' )
plt.plot( val_loss , label = 'Test Loss' )
plt.legend( loc = 'upper right' )
plt.ylabel( 'Cross Entropy' )
plt.ylim( [ 0 , max (plt.ylim( ) ) ] )
plt.title( 'Training and Test Loss' )
#and ...voilà, c'est magnifique!
plt.show( )

In [ ]:
acc = history.history [ 'accuracy' ]
val_acc = history.history [ 'val_accuracy' ]

In [ ]:
plt.figure( figsize = ( 8 , 8 ) )
plt.subplot( 2 , 1 , 1 )
plt.plot(acc, label = 'Training Accuracy' )
plt.plot( val_acc , label = 'Testing Accuracy' )
plt.legend( loc = 'lower right' )
plt.ylabel( 'Accuracy' )
plt.ylim( [min( plt.ylim( ) ) , 1] )
plt.title( 'Training and Test Accuracy' )

In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig , ax = plt.subplots ( 2 , 1 )
ax[0].plot(history.history[ 'loss' ] , color='b' , label = "Training loss")
ax[0].plot(history.history['val_loss'] , color='r' , label = "validation loss" ,axes = ax[ 0 ] )
legend = ax[0].legend( loc = 'best' , shadow = True )

ax[1].plot( history.history['accuracy'] , color = 'b' , label = "Training accuracy" )
ax[1].plot( history.history['val_accuracy'] , color = 'r' , label = "Validation accuracy" )
legend = ax[ 1 ].legend( loc = 'best' , shadow = True )

In [ ]:
# Look at confusion matrix 

def plot_confusion_matrix( cm , classes , normalize=False , title='Confusion matrix' , cmap = plt.cm.Blues ):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
#Y_pred = model.predict(X_val)
Y_pred = model.predict(tst_dt)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax( tst_dt , axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10))